# Tools and Routing

In [1]:
from aisModels_lc import GroqChatLLM
llm = GroqChatLLM()

Hello. How can I assist you today?
Hello! How are you today? Is there something I can help you with or would you like to chat?


In [2]:
import os

In [3]:
%%script true
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [4]:
from langchain.agents import tool

In [5]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [6]:
search.name

'search'

In [7]:
search.description

'Search for weather online'

In [8]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [9]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")


In [10]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [11]:
search.args

{'query': {'description': 'Thing to search for',
  'title': 'Query',
  'type': 'string'}}

In [12]:
search.run("sf")

'42f'

In [13]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [14]:
get_current_temperature.name

'get_current_temperature'

In [15]:
get_current_temperature.description

'Fetch current temperature for given coordinates.'

In [16]:
get_current_temperature.args

{'latitude': {'description': 'Latitude of the location to fetch weather data for',
  'title': 'Latitude',
  'type': 'number'},
 'longitude': {'description': 'Longitude of the location to fetch weather data for',
  'title': 'Longitude',
  'type': 'number'}}

In [17]:
from langchain.tools.render import format_tool_to_openai_function

In [18]:
format_tool_to_openai_function(get_current_temperature)

/tmp/ipykernel_12306/540789164.py:1: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  format_tool_to_openai_function(get_current_temperature)


{'name': 'get_current_temperature',
 'description': 'Fetch current temperature for given coordinates.',
 'parameters': {'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude'],
  'type': 'object'}}

In [19]:
get_current_temperature({"latitude": 24, "longitude": 67})

/tmp/ipykernel_12306/1081288606.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use invoke instead.
  get_current_temperature({"latitude": 24, "longitude": 67})


'The current temperature is 28.0°C'

In [20]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [21]:
search_wikipedia.name

'search_wikipedia'

In [22]:
search_wikipedia.description

'Run Wikipedia search and get page summaries.'

In [23]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'Run Wikipedia search and get page summaries.',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string'}},
  'required': ['query']}}

In [24]:
search_wikipedia({"query": "langchain"})

"Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: DataStax\nSummary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.\n\n\n\nPage: Retrieval-augmented generation\nSummary: Retrieval augmented generation (RAG) is a type of generative artificial intelligence that has information retrieval capabilities. It 

In [25]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [26]:
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [27]:
spec = OpenAPISpec.from_text(text)

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


AttributeError: 'super' object has no attribute 'parse_obj'

In [28]:
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

NameError: name 'spec' is not defined

In [29]:
pet_openai_functions

NameError: name 'pet_openai_functions' is not defined

In [30]:
#from langchain.chat_models import ChatOpenAI

In [28]:
model = llm #ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

In [29]:
model.invoke("what are three pets names")

AIMessage(content='Here are three pet name suggestions:\n\n1. Luna\n2. Max\n3. Bella', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 40, 'total_tokens': 59, 'completion_time': 0.076, 'prompt_time': 0.010976359, 'queue_time': 0.005716819, 'total_time': 0.086976359}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b3ae7e594e', 'finish_reason': 'stop', 'logprobs': None}, id='run-b6537680-c560-4d3c-b813-d4a6f9766562-0', usage_metadata={'input_tokens': 40, 'output_tokens': 19, 'total_tokens': 59})

In [30]:
model.invoke("tell me about pet with id 42")

AIMessage(content="I don't have any information about a pet with ID 42. I'm a large language model, I don't have access to real-time data or specific information about individual pets. If you could provide more context or details about the pet with ID 42, I'd be happy to try and help you.", response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 43, 'total_tokens': 107, 'completion_time': 0.256, 'prompt_time': 0.00996735, 'queue_time': 0.0052196280000000005, 'total_time': 0.26596735}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b3ae7e594e', 'finish_reason': 'stop', 'logprobs': None}, id='run-8d1c23ea-1575-4142-871b-3e69d3e42cb7-0', usage_metadata={'input_tokens': 43, 'output_tokens': 64, 'total_tokens': 107})

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [31]:
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = llm.bind(functions=functions)

In [32]:
model.invoke("what is the weather in sf right now at Lat: 25, Long: 67")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude": 25, "longitude": 67}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 319, 'total_tokens': 339, 'completion_time': 0.080250975, 'prompt_time': 0.085001153, 'queue_time': 1.936932815, 'total_time': 0.165252128}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_9260b4bb2e', 'finish_reason': 'function_call', 'logprobs': None}, id='run-f41142cd-b9cd-4d47-9524-180d70a4b446-0', usage_metadata={'input_tokens': 319, 'output_tokens': 20, 'total_tokens': 339})

In [33]:
model.invoke("what is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query": "langchain"}', 'name': 'search_wikipedia'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 305, 'total_tokens': 321, 'completion_time': 0.064, 'prompt_time': 0.077798512, 'queue_time': 0.12909621999999998, 'total_time': 0.141798512}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_5c5d1b5cfb', 'finish_reason': 'function_call', 'logprobs': None}, id='run-543a33dd-eeca-4657-9211-bee30840903e-0', usage_metadata={'input_tokens': 305, 'output_tokens': 16, 'total_tokens': 321})

In [34]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [35]:
chain.invoke({"input": "what is the weather in sf right now"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude": 37.7749, "longitude": -122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 316, 'total_tokens': 342, 'completion_time': 0.104, 'prompt_time': 0.079582449, 'queue_time': 0.0006372000000000044, 'total_time': 0.183582449}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b6828be2c9', 'finish_reason': 'function_call', 'logprobs': None}, id='run-87323128-2ce5-4fb0-9ef4-53fde774ae86-0', usage_metadata={'input_tokens': 316, 'output_tokens': 26, 'total_tokens': 342})

In [36]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [37]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [38]:
result = chain.invoke({"input": "what is the weather in sf right now"})

In [39]:
type(result)

langchain_core.agents.AgentActionMessageLog

In [40]:
result.tool

'get_current_temperature'

In [41]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [42]:
get_current_temperature(result.tool_input)

'The current temperature is 14.2°C'

In [43]:
result = chain.invoke({"input": "hi!"})

In [44]:
type(result)

langchain_core.agents.AgentFinish

In [45]:
result.return_values

{'output': 'What can I help you with today?'}

In [46]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [47]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [48]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})

In [49]:
result

'The current temperature is 14.2°C'

In [50]:
result = chain.invoke({"input": "What is langchain?"})

In [51]:
result

"Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: DataStax\nSummary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.\n\n\n\nPage: Retrieval-augmented generation\nSummary: Retrieval augmented generation (RAG) is a type of generative artificial intelligence that has information retrieval capabilities. It 

In [52]:
chain.invoke({"input": "hi!"})

'Hello. What can I help you with today?'